In [1]:
from theano import *
from lasagne.layers import EmbeddingLayer, InputLayer, get_output
import lasagne
import lasagne.layers
import theano.tensor as T
import theano
import numpy as np

Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
from wordvecs import WordVectors

wordvectors = WordVectors(fname="../enwiki-20141208-pages-articles-multistream-links-output4.bin", negvectors=True)

from sentiment_sents import Sentiment

# just load the sentences from the CNN system
sentiment = Sentiment("prevwork/CNN_sentence/")

In [3]:
len(wordvectors.vectors)

6091842

In [4]:
l_in = InputLayer((3,))

In [5]:
W = np.arange(3*5).reshape((3,5)).astype('float32')

In [6]:
l1 = EmbeddingLayer(l_in, input_size=3, output_size=5, W=W)

In [7]:
x = T.imatrix()

In [8]:
output = get_output(l1, x)

/home/matthew/.virtualenvs/nlp-convnet/lib/python2.7/site-packages/lasagne/layers/helper.py:69: UserWarning: get_all_layers() has been changed to return layers in topological order. The former implementation is still available as get_all_layers_old(), but will be removed before the first release of Lasagne. To ignore this warning, use `warnings.filterwarnings('ignore', '.*topo.*')`.
  warnings.warn("get_all_layers() has been changed to return layers in "


In [9]:
f = theano.function([x], output)

In [10]:
x_test = np.array([[0,2],[1,2]]).astype('int32')

In [11]:
f(x_test)

array([[[  0.,   1.,   2.,   3.,   4.],
        [ 10.,  11.,  12.,  13.,  14.]],

       [[  5.,   6.,   7.,   8.,   9.],
        [ 10.,  11.,  12.,  13.,  14.]]], dtype=float32)

Going to attempt to make the simple linear reg working, just to experiment with theano

In [12]:
np.random.randn(10)

array([-0.61099448, -0.11652368,  2.05125713,  0.38902261, -0.90401012,
        0.20400009, -1.50504069,  3.3007176 ,  0.13135793,  1.44839975])

In [13]:
nfeats = 10


x = T.matrix('x')
y = T.vector('y')
w = theano.shared(np.random.randn(nfeats),name='w')
b = theano.shared(0.0,name='b')

In [14]:
p_1 = 1 / (1 + T.exp(-T.dot(x,w) - b))
prediction = p_1 > 0.5
xent = -y * T.log(p_1) - (1-y) * T.log(1-p_1)
cost = xent.mean() + .01 * (w**2).sum()
gw, gb = T.grad(cost, [w,b])

In [15]:
w.get_value().shape

(10,)

In [16]:
train = theano.function(inputs=[x,y],outputs=[prediction,xent], updates=((w, w-0.1*gw),(b,b - 0.1*gb)))
predict = theano.function(inputs=[x], outputs=prediction)

In [17]:
D = (np.random.randn(400, nfeats), np.random.randint(size=400, low=0, high=2))

In [18]:
theano.printing.debugprint(train)

Elemwise{Composite{GT(scalar_sigmoid(i0), i1)}} [@A] ''   15
 |Elemwise{neg,no_inplace} [@B] ''   13
 | |Elemwise{Composite{((-i0) - i1)}}[(0, 0)] [@C] ''   10
 |   |CGemv{inplace} [@D] ''   8
 |   | |AllocEmpty{dtype='float64'} [@E] ''   5
 |   | | |Shape_i{0} [@F] ''   1
 |   | |   |x [@G]
 |   | |TensorConstant{1.0} [@H]
 |   | |x [@G]
 |   | |w [@I]
 |   | |TensorConstant{0.0} [@J]
 |   |InplaceDimShuffle{x} [@K] ''   0
 |     |b [@L]
 |TensorConstant{(1,) of 0.5} [@M]
Elemwise{Composite{((i0 * scalar_softplus(i1)) - (i2 * i3 * scalar_softplus(i4)))}}[(0, 1)] [@N] ''   17
 |y [@O]
 |Elemwise{Composite{((-i0) - i1)}}[(0, 0)] [@C] ''   10
 |TensorConstant{(1,) of -1.0} [@P]
 |Elemwise{sub,no_inplace} [@Q] ''   4
 | |TensorConstant{(1,) of 1.0} [@R]
 | |y [@O]
 |Elemwise{neg,no_inplace} [@B] ''   13
CGemv{inplace} [@S] ''   19
 |w [@I]
 |TensorConstant{-0.1} [@T]
 |InplaceDimShuffle{1,0} [@U] 'x.T'   2
 | |x [@G]
 |Elemwise{Composite{((i0 * i1 * i2 * i3) + ((scalar_sigmoid((-i4)) * i5

In [19]:
for i in xrange(10000):
    pred, err = train(D[0], D[1])

In [20]:
err.mean()

0.67402486455032262

In [21]:
D[0][1]

array([ 0.82726345, -1.34371496, -2.04404689, -0.94441741, -0.11134401,
        0.21794162,  0.3532185 , -0.59029774, -0.31446545,  0.52206763])

In [22]:
w.get_value(), b.get_value()

(array([ 0.05472259, -0.12900095, -0.19193356,  0.05842622, -0.06939324,
        -0.01532577,  0.03732123, -0.18237269,  0.03827691,  0.09780588]),
 array(0.16938777211978998))

In [23]:
predict([D[0][1]])

array([1], dtype=int8)

In [24]:
D[1][1]

1

going to see if we can make a few layers of a cnn given that went through the example of log reg from the tutorial

In [25]:
def make_context(sentence):
    ret = []
    for w in sentence.lower().split():
        ret.append(wordvectors[w])
    return ret

In [26]:
class SentimentExp(object):
    
    def __init__(self, train_X, train_Y, wordvecs=wordvectors):
        self.train_X = train_X
        self.train_Y = theano.shared(np.array(train_Y, 'int32'))
        self.wordvecs = wordvecs
        
        self.input_size = 10
        self.batch_size = 10
        
        self.learning_rate = .01
        self.momentum = .9
        
        self.train_X_rep = theano.shared(lasagne.utils.floatX([[self.getRep(x)] for x in self.train_X]))
                
        self._setup()
        
    def getRep(self, sent):
        ret = []
        for i in xrange(self.input_size):
            if i < len(sent):
                ret.append(self.wordvecs[sent[i]])
            else:
                ret.append(np.zeros(self.wordvecs.vector_size))
        return np.matrix(ret).reshape((1, self.input_size, self.wordvecs.vector_size))
    
    def _setup(self):
        self.y_batch = T.ivectors('y')
        self.x_batch = T.tensor4('x')  # have to match the dimention of the input layer
        
        self.batch_index = T.iscalar('batch_index')
        self.batch_slice = slice(self.batch_index * self.batch_size, (self.batch_index + 1) * self.batch_size)
        
        self.input_l = InputLayer(shape=(self.batch_size, 1, self.input_size, self.wordvecs.vector_size), name='x')
        self.conv1_l = lasagne.layers.Conv2DLayer(
            self.input_l,
            num_filters=32, 
            filter_size=5,
            name='conv1'
        )
        #self.conv2_l = lasagne.layers.Conv2DLayer(
        #   self.conv1_l,
        #    num_filters=32,
        #    filter_size=5,
        #)
        self.output_l = lasagne.layers.DenseLayer(self.conv1_l, num_units=1, name='dens2')
        
        self.output_v = lasagne.layers.get_output(self.output_l, self.x_batch)
        self.loss_v = lasagne.objectives.binary_crossentropy(self.output_v, self.y_batch).mean()
        
        self.output_test = lasagne.layers.get_output(self.output_l, self.x_batch, deterministic=True)
        self.loss_eval = lasagne.objectives.binary_crossentropy(self.output_test, self.y_batch)
        
        self.all_params = lasagne.layers.get_all_params(self.output_l)
        self.updates = lasagne.updates.nesterov_momentum(self.loss_v, self.all_params, 
                                                         self.learning_rate, self.momentum)
        
        self.prediction = T.argmax(self.output_v)
        self.accuract = T.mean(T.eq(self.prediction, self.y_batch), dtype=theano.config.floatX)
        
        self.iter_train = theano.function(
            [self.batch_index], self.loss_v,
            updates=self.updates,
            givens={
                self.x_batch: self.train_X_rep[self.batch_slice],
                self.y_batch: self.train_Y[self.batch_slice]
            },
            mode='DebugMode'
        )
        
        self.test_valid = theano.function(
            [self.x_batch, self.y_batch], [self.loss_eval, self.prediction, self.accuract],
        )
            
    def train(self):
        pass
            
experiment = SentimentExp(sentiment.train_X, sentiment.train_Y)

/home/matthew/.virtualenvs/nlp-convnet/lib/python2.7/site-packages/lasagne/init.py:86: UserWarning: The uniform initializer no longer uses Glorot et al.'s approach to determine the bounds, but defaults to the range (-0.01, 0.01) instead. Please use the new GlorotUniform initializer to get the old behavior. GlorotUniform is now the default for all layers.
  warnings.warn("The uniform initializer no longer uses Glorot et al.'s "


In [27]:
theano.config.optimizer = 'None'  # disable the optimizer so that we can maybe get more debugging information
theano.config.exception_verbosity = 'high'

In [28]:
experiment.input_l.shape, experiment.train_X_rep[experiment.batch_slice].eval({experiment.batch_index:0}).shape

((10, 1, 10, 100), (10, 1, 10, 100))

In [29]:
experiment.train_Y[experiment.batch_slice].eval({experiment.batch_index:0}).shape

(10,)

In [30]:
experiment.iter_train(0)

/home/matthew/.virtualenvs/nlp-convnet/lib/python2.7/site-packages/theano/tensor/nnet/conv.py:771: ComplexWarning: Casting complex values to real discards the imaginary part
  1, val, bval, 0)


ValueError: An optimization (probably local_mul_canonizer) inserted an apply node that raise an error.
The information we have about this optimizations is:Elemwise{true_div,no_inplace}.0
  Elemwise{true_div,no_inplace} [@A] ''   
   |Elemwise{mul,no_inplace} [@B] ''   
   | |TensorConstant{(1, 1) of 2.0} [@C]
   | |Elemwise{mul,no_inplace} [@D] ''   
   | | |TensorConstant{(1, 1) of -1.0} [@E]
   | | |Elemwise{second,no_inplace} [@F] ''   
   | |   |DimShuffle{x,x} [@G] ''   
   | |   | |Elemwise{true_div,no_inplace} [@H] ''   
   | |   |Elemwise{true_div} [@I] ''   
   | |     |Elemwise{true_div} [@J] ''   
   | |     |DimShuffle{x,x} [@K] ''   
   | |DimShuffle{x,0} [@L] ''   
   |   |Subtensor{int32:int32:} [@M] ''   
   |     |<TensorType(int32, vector)> [@N]
   |     |ScalarFromTensor [@O] ''   
   |     | |Elemwise{mul,no_inplace} [@P] ''   
   |     |ScalarFromTensor [@Q] ''   
   |       |Elemwise{mul,no_inplace} [@R] ''   
   |Elemwise{add,no_inplace} [@S] ''   
     |Elemwise

In [ ]:
from helpers import *

In [ ]:
batch_index = T.iscalar('batch_index')
X_batch = T.matrix('x.input')
y_batch = T.ivector('y')
slice(batch_index * 100, (batch_index + 1) * 100)

In [ ]:
experiment.

In [ ]:
experiment.input.name

In [ ]:
l_in = lasagne.layers.InputLayer(shape=(10,1,10,10))
l_conv1 = lasagne.layers.Conv2DLayer(l_in, num_filters=32, filter_size=(5,5))
l_out = lasagne.layers.DenseLayer(l_conv1, num_units=1)

In [ ]:
lasagne.layers.get_output(l_out)

In [ ]:
sentiment.train_X[0]

In [ ]:
theano.

In [ ]:
X_batch

In [ ]:
theano.shared(np.array(sentiment.train_Y, 'int32'))

In [ ]:
T.ivector()